<a href="https://colab.research.google.com/github/robingenz/htwg-machine-learning-exercises/blob/main/exercises/01_Explorative_Analyse_und_PCA/01-explorative-analyse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementierung der Hauptkomponentenanalyse

## Setup

### Einbinden von Paketen

Häufig verwendete Pakete einbinden und Versionsinformationen ausgeben:

In [39]:
import numpy as np
import pandas as pd
%matplotlib inline

np.__version__, pd.__version__, 

('1.23.4', '1.5.0')

In [40]:
%load_ext version_information
%version_information

The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information


Software versions
Python 3.10.8 64bit [GCC 10.2.1 20210110]
IPython 8.5.0
OS Linux 5.10.104 linuxkit aarch64 with glibc2.31
Fri Oct 28 17:27:14 2022 UTC

### Einlesen der Daten

Datensatz `housing.data` herunterladen:

In [41]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data'
cols = ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B',
        'LSTAT','TGT']
dataFrame = pd.read_csv(url, sep=' ', skipinitialspace=True, header=None, names=cols, 
                     index_col=False)
dateDownloaded = !date #Calling Linux
dateDownloaded

['Fri Oct 28 17:27:15 UTC 2022']

In [42]:
dataFrame.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,TGT
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


## Aufgaben

### Aufgabe a)

Erstellen der Methode `pca(dataFrame: pd.DataFrame)` zur Durchführung der Hauptkomponentenanalyse:

In [43]:
from scipy import stats

def pca(dataFrame: pd.DataFrame):
    # 1. Zentrierung
    for col in dataFrame.columns:
        dataFrame[col] = dataFrame[col] - dataFrame[col].mean()
    # 2. Normierung
    dataFrame = dataFrame.apply(stats.zscore)
    # 3. Erstellen einer Daten- und Designmatrix
    X = dataFrame.to_numpy()
    # 4.-7. Berechnung der Singulärwertzerlegung
    U, D, V = np.linalg.svd(X, full_matrices=False)
    # U: Unitäre Matrix mit linken singulären Vektoren als Spalten der Form (M,K)
    # D: Enthält die singulären Vektoren
    # V: Unitäre Matrix mit rechten singulären Vektoren als Reihen der Form (K,N)
    return U, D, V

### Aufgabe b)

Nun wird die zuvor definierte Methode am Datensatz getestet:

In [56]:
dataFrameWithOutTgt = dataFrame.drop('TGT', axis=1)
U, D, V = pca(dataFrameWithOutTgt)

n = U.shape[0]

dataFrameResults = pd.DataFrame(D, columns=['Singular Value'])
dataFrameResults['Eigenvalue'] = dataFrameResults['Singular Value'] ** 2
dataFrameResults['Varianz'] = dataFrameResults['Singular Value'] / (n - 1)
dataFrameResults['Explained Variance (%)'] = (dataFrameResults['Varianz'] / (dataFrameResults['Varianz'].sum())) * 100
dataFrameResults['Cumulative Explained Variance (%)'] = dataFrameResults['Explained Variance (%)'].cumsum()
dataFrameResults['Error (%)'] = round(100 - dataFrameResults['Cumulative Explained Variance (%)'], 3)
dataFrameResults

,Singular Value,Eigenvalue,Varianz,Explained Variance (%),Cumulative Explained Variance (%),Error (%)
0,55.679309,3100.185506,0.110256,22.731479,22.731479,77.269
1,26.930229,725.237212,0.053327,10.994460,33.725939,66.274
2,25.075168,628.764037,0.049654,10.237118,43.963057,56.037
3,20.831059,433.933005,0.041250,8.504430,52.467486,47.533
4,20.552782,422.416864,0.040699,8.390821,60.858308,39.142
5,18.238641,332.648031,0.036116,7.446057,68.304364,31.696
6,16.458742,270.890180,0.032592,6.719400,75.023764,24.976
7,14.157162,200.425241,0.028034,5.779763,80.803528,19.196
8,11.837792,140.133325,0.023441,4.832864,85.636392,14.364
9,10.556531,111.440339,0.020904,4.309780,89.946171,10.054


> Achtung: die Diagonalelemente von $D$ müssen dafür quadriert und durch n − 1 geteilt werden. Warum?

Dies ist notwendig, da die singulären Werte als Wurzel der Eigenwerte definiert sind (siehe https://math.stackexchange.com/a/127512).  
Durch `n - 1` muss geteilt werden, damit man die Varianz erhält (siehe https://stats.stackexchange.com/a/134283).

> Wieviele Dimensionen können Sie weglassen, wenn Sie 10%, 5% und 1% Fehler bei der Dimensionsreduktion zulassen?

`10%`: 3 Dimensionen  
`5%`: 2 Dimensionen  
`1%`: 1 Dimension  